In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [2]:
x = torch.load("/data/3d/ixi-data.pt")
y = torch.load("/data/3d/ixi-labels.pt")
n = x.shape[0]
y = torch.reshape(y,(n,1))
print(x.shape,y.shape)
print(x.dtype,y.dtype)

torch.Size([536, 3, 224, 224]) torch.Size([536, 1])
torch.float32 torch.float32


In [3]:
fcp_x = torch.load("/data/3d/fcp-data.pt")
fcp_y = torch.load("/data/3d/fcp-labels.pt").float()
n = fcp_x.shape[0]
fcp_y = torch.reshape(fcp_y,(n,1))
print(fcp_x.shape,fcp_y.shape)
print(fcp_x.dtype,fcp_y.dtype)

torch.Size([1033, 3, 224, 224]) torch.Size([1033, 1])
torch.float32 torch.float32


In [4]:
adni_x = torch.load('/data/3d/adni-data.pt')
adni_y = torch.load('/data/3d/adni-labels.pt').float()
adni_y = torch.reshape(adni_y, (adni_y.shape[0], 1))
print(adni_x.shape, adni_y.shape)
print(adni_x.dtype, adni_y.dtype)

torch.Size([570, 3, 224, 224]) torch.Size([570, 1])
torch.float32 torch.float32


In [5]:
#Normalizing the data (according to the stats provided) before feeding into the pretrained model
# fcp_x_normalized = torch.Tensor(fcp_x.shape)
# for i in range(n):
#     fcp_x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]).__call__(fcp_x[i,:,:])


In [6]:
x = torch.cat((x, fcp_x, adni_x))
y = torch.cat((y, fcp_y, adni_y))
print(x.dtype, y.dtype)

torch.float32 torch.float32


In [7]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [8]:
dataset = utils.TensorDataset(x, y)
n = x.shape[0]
print('Total Data:', n)

Total Data: 2139


In [9]:
train_indices = [i for i in range(n) if (i % 5 == 0 or i % 5 == 2 or i % 5 == 4)]
val_indices = [i for i in range(n) if i % 5 == 3]
test_indices = [i for i in range(n) if i not in train_indices and i not in val_indices]

print('Training Data:', len(train_indices))
print('Validation Data:', len(val_indices))
print('Test Data:', len(test_indices))

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(train_indices),num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(val_indices),num_workers=4)

Training Data: 1283
Validation Data: 428
Test Data: 428


In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))

            epoch_loss = running_loss / n
            epoch_offset = running_offset / n

            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [12]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 268.7251 Diff: 9.5412
val Loss: 32.7319 Diff: 1.9731

Epoch 1/9
----------
train Loss: 110.5941 Diff: 6.1553
val Loss: 31.8473 Diff: 1.7752

Epoch 2/9
----------
train Loss: 88.2281 Diff: 5.3993
val Loss: 29.9023 Diff: 1.9298

Epoch 3/9
----------
train Loss: 82.8230 Diff: 5.1688
val Loss: 50.5370 Diff: 2.4677

Epoch 4/9
----------
train Loss: 74.5760 Diff: 4.9289
val Loss: 25.4814 Diff: 1.7020

Epoch 5/9
----------
train Loss: 66.4270 Diff: 4.5685
val Loss: 25.8077 Diff: 1.6137

Epoch 6/9
----------
train Loss: 53.6232 Diff: 4.1240
val Loss: 20.8035 Diff: 1.4904

Epoch 7/9
----------
train Loss: 48.2163 Diff: 3.8789
val Loss: 19.8810 Diff: 1.4628

Epoch 8/9
----------
train Loss: 43.4534 Diff: 3.6725
val Loss: 18.7653 Diff: 1.3975

Epoch 9/9
----------
train Loss: 42.3070 Diff: 3.6023
val Loss: 19.0811 Diff: 1.4110

Training complete in 3m 24s
Best val Diff: 1.397510


In [59]:
def check_error(data,labels,model):
    print('Checking error on test set')   
    total_abs_err = 0.0
    num_samples = labels.shape[0]
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device,dtype=dtype)
        scores = model(data)
        predicted_ages = scores
        total_abs_err += (torch.abs(labels - scores).sum())
        mean_abs_err = float(total_abs_err) / num_samples
        print('Got error %.2f' % (mean_abs_err))
        
        large_difference = []
        for i in range(num_samples):
            if abs(scores[i].item() - labels[i].item()) > 10:
                large_difference.append(test_indices[i])
                
        print('Large difference items', large_difference)

In [60]:
check_error(x[test_indices,:], y[test_indices],model_ft)

Checking error on test set
Got error 7.40
Large difference items [16, 36, 46, 76, 86, 96, 111, 126, 141, 146, 151, 156, 191, 196, 201, 206, 211, 221, 226, 246, 256, 266, 271, 276, 291, 296, 301, 306, 316, 326, 336, 341, 346, 361, 376, 381, 396, 401, 406, 421, 426, 436, 441, 446, 451, 461, 466, 476, 491, 496, 506, 511, 516, 526, 536, 541, 551, 561, 571, 581, 591, 601, 611, 626, 631, 636, 641, 726, 731, 791, 821, 831, 836, 856, 936, 1141, 1156, 1166, 1181, 1206, 1211, 1216, 1221, 1236, 1246, 1251, 1256, 1261, 1266, 1271, 1276, 1481, 1486, 1496, 1501, 1521, 1531, 1541, 1546, 1621, 1626, 1641, 1751, 1756, 1776, 1781, 1821, 1831, 1871, 1886, 1956, 2021, 2116, 2126]
